In [41]:
import pandas as pd
import gmaps
import gmaps.datasets
import requests
import json

# Import API key
from config import g_key

In [42]:
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Torbay,CA,2022-04-13 18:14:46,47.6666,-52.7314,37.27,91,100,20.71
1,1,Khatanga,RU,2022-04-13 18:14:46,71.9667,102.5000,14.79,95,100,9.62
2,2,Victoria,HK,2022-04-13 18:14:15,22.2855,114.1577,78.33,85,69,5.01
3,3,Ushuaia,AR,2022-04-13 18:09:49,-54.8000,-68.3000,56.86,62,40,20.71
4,4,Longyearbyen,SJ,2022-04-13 18:14:05,78.2186,15.6401,19.24,79,40,5.75


In [43]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [44]:
gmaps.configure(api_key=g_key)

In [45]:
max_temp = city_data_df['Max Temp']
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [46]:
# heatmap of temperature
# get the latitude and longitude
locations = city_data_df[['Lat', 'Lng']]
# get the maximum temperature
max_temp = city_data_df['Max Temp']
# assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# add the heatmap layer
fig.add_layer(heat_layer)
# call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
# heatmap of humidity
locations = city_data_df[['Lat', 'Lng']]
# get the humidity
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
locations = city_data_df[['Lat', 'Lng']]

clouds = city_data_df['Cloudiness']

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [49]:
locations = city_data_df[['Lat', 'Lng']]

winds = city_data_df['Wind Speed']

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=winds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
# ask the customer to add minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [51]:
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<=max_temp)&(city_data_df['Max Temp']>=min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Victoria,HK,2022-04-13 18:14:15,22.2855,114.1577,78.33,85,69,5.01
5,5,Quatre Cocos,MU,2022-04-13 18:14:48,-20.2078,57.7625,79.09,84,48,20.60
7,7,Rikitea,PF,2022-04-13 18:13:55,-23.1203,-134.9692,78.80,69,5,18.10
9,9,Vaini,TO,2022-04-13 18:13:57,-21.2000,-175.2000,75.36,94,40,4.61
17,17,Arraial Do Cabo,BR,2022-04-13 18:14:53,-22.9661,-42.0278,87.76,74,0,12.66
22,22,Puerto Ayora,EC,2022-04-13 18:14:01,-0.7393,-90.3518,80.56,78,73,10.27
25,25,Houma,US,2022-04-13 18:14:57,29.5958,-90.7195,84.00,75,0,20.71
27,27,Avarua,CK,2022-04-13 18:12:45,-21.2078,-159.7750,75.25,100,75,1.14
28,28,Yulara,AU,2022-04-13 18:14:58,-25.2406,130.9889,80.44,28,81,6.91
32,32,Padang,ID,2022-04-13 18:15:00,-0.9492,100.3543,78.91,86,100,2.55


In [52]:
# create a dataframe called hotel_df to store hotels names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Victoria,HK,78.33,22.2855,114.1577,
5,Quatre Cocos,MU,79.09,-20.2078,57.7625,
7,Rikitea,PF,78.80,-23.1203,-134.9692,
9,Vaini,TO,75.36,-21.2000,-175.2000,
17,Arraial Do Cabo,BR,87.76,-22.9661,-42.0278,
22,Puerto Ayora,EC,80.56,-0.7393,-90.3518,
25,Houma,US,84.00,29.5958,-90.7195,
27,Avarua,CK,75.25,-21.2078,-159.7750,
28,Yulara,AU,80.44,-25.2406,130.9889,
32,Padang,ID,78.91,-0.9492,100.3543,


In [53]:
# set the parameters to search for a hotel
params = {
    'radius': 15000,
    'type': 'lodging',
    'key': g_key
}

In [78]:
# iterate through the dataframe
for index, row in hotel_df.iterrows():
    # get the latitude/longitutde 
    lat = row['Lat']
    lng = row['Lng']

    # add the latitude and longitude to location key for the params dictionary
    params['location'] = f"{lat},{lng}"

    # use the search term: "lodging" and our latitude and longitude
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    # make request and get the json data from the search
    hotels = requests.get(base_url, params=params).json()
    # grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found...skipping.")
        
print(json.dumps(hotels['results'][0]['name'], indent=2))

Hotel not found...skipping.
Hotel not found...skipping.
"The Old House"


In [81]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Victoria,HK,78.33,22.2855,114.1577,Mini Hotel Central
5,Quatre Cocos,MU,79.09,-20.2078,57.7625,One&Only Le Saint Géran
7,Rikitea,PF,78.80,-23.1203,-134.9692,People ThankYou
9,Vaini,TO,75.36,-21.2000,-175.2000,Little Italy Hotel
17,Arraial Do Cabo,BR,87.76,-22.9661,-42.0278,Pousada Porto Praia
22,Puerto Ayora,EC,80.56,-0.7393,-90.3518,Finch Bay Galapagos Hotel
25,Houma,US,84.00,29.5958,-90.7195,Courtyard by Marriott Houma
27,Avarua,CK,75.25,-21.2078,-159.7750,Little Polynesian Resort
28,Yulara,AU,80.44,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
32,Padang,ID,78.91,-0.9492,100.3543,Mercure Padang


In [84]:
# add a heatmap of temperature for the vacation spots
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [87]:
info_box_template="""
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} F</dd>
</dl>
"""
# store the dataframe row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [88]:
# add a heatmap of temperature for the vacation spots and a pop-up marker for each city
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))